---  
---  
# Prepare Training and Test Data

In [1]:
import sys, os
sys.path.append("/localdata2/jent_so/LM_GenderBias")
sys.path.append("/localdata2/jent_so/LM_GenderBias/resources")

import pandas as pd
#import xlsxwriter
import data_masking as masking

import logging
# create logger
logger_data_prep = logging.getLogger('simple_example')
logger_data_prep.setLevel(logging.INFO)
#logger_data_prep.config(filename='example.log', encoding='utf-8')

INFO:root:imported term dicts: prons_m2f, prons_f2m, terms_m2f, terms_f2m; and sets; all_prons and all_terms
INFO:root:successfully imported the latest version of data_masking.


The following is only done once in the very beginning. The IDs are ambigious in the original data set as each number occurs in positive and negative data sets in test and training condition. Therefore I **clean the tables and add unique IDs**


In [4]:
df_train


,ID,text,label
0,train_pos_0,bromwell high is a cartoon comedy it ran at th...,pos
1,train_pos_1,if you like adult comedy cartoons like south p...,pos
2,train_pos_2,bromwell high is nothing short of brilliant ex...,pos
3,train_pos_3,all the world s a stage and its people actors ...,pos
4,train_pos_4,futz is the only show preserved from the exper...,pos
...,...,...,...
24995,train_neg_12495,ok i love bad horror i especially love horror ...,neg
24996,train_neg_12496,to be brutally honest i loved watching severed...,neg
24997,train_neg_12497,i m sure that the folks on the texas louisiana...,neg
24998,train_neg_12498,this film has the kernel of a really good stor...,neg


In [2]:
df_train = pd.read_pickle('IMDB_train')
df_test = pd.read_pickle('IMDB_test')
#
df_train_ = pd.read_pickle('IMDB_l_train')
df_test_ = pd.read_pickle('IMDB_l_test')

df_train_.head(2)

,ID,text,label,count_table,count_total,count_table_weat,count_weat,count_prons,len,text_all_M,text_all_F,text_all_N,text_weat_M,text_weat_F,text_weat_N,text_pro_M,text_pro_F,text_pro_N
0,train_pos_0,bromwell high is a cartoon comedy it ran at th...,pos,"{'grandmotherly': 0, 'adulteress': 0, 'spokesm...",0,"{'his': 0, 'grandmother': 0, 'brothers': 0, 'a...",0,0,141,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...
1,train_pos_1,if you like adult comedy cartoons like south p...,pos,"{'grandmotherly': 0, 'adulteress': 0, 'spokesm...",2,"{'his': 0, 'grandmother': 0, 'brothers': 0, 'a...",1,0,119,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...


### Step 1: Gender neutral data sets for training

- condition 1: **remove** gender terms - `_N`
- condition 2: **replace** and use both - `_M + _F`  

- three different dicts: (1) all = big dict terms (2) weat = dict only contains WEAT target terms (3) prons = dict only contains pronouns as target terms  
`text_all_M` | `text_all_F` | `text_all_N`  
`text_weat_M` | `text_weat_F` | `text_weat_N`  
`text_pron_M` | `text_pron_F` | `text_pron_N`

In [13]:
df_train_.head(2)

,ID,text,label,count_table,count_total,count_table_weat,count_weat,count_prons,len,text_all_M,text_all_F,text_all_N,text_weat_M,text_weat_F,text_weat_N,text_pro_M,text_pro_F,text_pro_N
0,train_pos_0,bromwell high is a cartoon comedy it ran at th...,pos,"{'grandmotherly': 0, 'adulteress': 0, 'spokesm...",0,"{'his': 0, 'grandmother': 0, 'brothers': 0, 'a...",0,0,141,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...,bromwell high is a cartoon comedy it ran at th...
1,train_pos_1,if you like adult comedy cartoons like south p...,pos,"{'grandmotherly': 0, 'adulteress': 0, 'spokesm...",2,"{'his': 0, 'grandmother': 0, 'brothers': 0, 'a...",1,0,119,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...,if you like adult comedy cartoons like south p...


Safe training and test dataframes for different training conditions.  
neutral and mixed 

In [3]:
# neutral
for spec in ['_all', '_pro', '_weat']:
    df_train_[['ID', 'text'+spec+'_N', 'label']].to_pickle('IMDB_training/IMDB_N'+spec+'_train')
    df_test_[['ID', 'text'+spec+'_N', 'label']].to_pickle('IMDB_training/IMDB_N'+spec+'_test')

# mixed M+F
for spec in ['_all', '_pro', '_weat']: 
    m_tr = df_train_[['ID', 'text'+spec+'_M', 'label']].rename(columns={'text'+spec+'_M': 'text'})
    f_tr = df_train_[['ID', 'text'+spec+'_F', 'label']].rename(columns={'text'+spec+'_F': 'text'})
    tr = m_tr.append(f_tr)
    tr.to_pickle('IMDB_training/IMDB_mix' + spec + '_train') 
    
    m_te = df_test_[['ID', 'text'+spec+'_M', 'label']].rename(columns={'text'+spec+'_M': 'text'})
    f_te = df_test_[['ID', 'text'+spec+'_F', 'label']].rename(columns={'text'+spec+'_F': 'text'})
    te = m_te.append(f_te)
    te.to_pickle('IMDB_training/IMDB_mix' + spec + '_test') 
    
    print(tr.shape, te.shape)    

(50000, 3) (50000, 3)
(50000, 3) (50000, 3)
(50000, 3) (50000, 3)


### Create Data Sets with no only samples that do not contain any term of the dict
dicts are again pron, weat, alll

In [23]:
# no term sample

df_train_no_pron = df_train_[df_train_['count_total'] == 0][['ID', 'text', 'label']]
print(df_train_no_pron.shape)
df_test_no_pron = df_test_[df_test_['count_total'] == 0][['ID', 'text', 'label']]
print(df_test_no_pron.shape)

df_train_no_weat = df_train_[df_train_['count_weat'] == 0][['ID', 'text', 'label']]
print(df_train_no_weat.shape)
df_test_no_weat = df_test_[df_test_['count_weat'] == 0][['ID', 'text', 'label']]
print(df_test_no_weat.shape)

df_train_no_all = df_train_[df_train_['count_prons'] == 0][['ID', 'text', 'label']]
print(df_train_no_all.shape)
df_test_no_all = df_test_[df_test_['count_prons'] == 0][['ID', 'text', 'label']]
print(df_test_no_all.shape)


df_train_no_pron.to_pickle('IMDB_training/IMDB_no_pron_train')
df_test_no_pron.to_pickle('IMDB_training/IMDB_no_pron_test')
df_train_no_weat.to_pickle('IMDB_training/IMDB_no_weat_train')
df_test_no_weat.to_pickle('IMDB_training/IMDB_no_weat_test')
df_train_no_all.to_pickle('IMDB_training/IMDB_no_all_train')
df_test_no_all.to_pickle('IMDB_training/IMDB_no_all_test')

(4178, 3)
(4222, 3)
(6488, 3)
(6668, 3)
(8322, 3)
(8655, 3)


### Create Data Sets with no only samples that do contain a minimal number of term of the dict
dicts are again pron, weat, all  
minimal number should most likely be 1 for IMDB

In [4]:
min_term_count = 1
df_train__ = df_train_.rename(columns={'count_total': 'count_all', 'count_prons': 'count_pro'})
df_test__ = df_test_.rename(columns={'count_total': 'count_all', 'count_prons': 'count_pro'})

# df_train_MIN_pro = df_train_[df_train_['count_total'] >= min_term_count]
# df_test_MIN_pro = df_test_[df_test_['count_total'] >= min_term_count]

# df_train_MIN_weat = df_train_[df_train_['count_weat'] >= min_term_count]
# df_test_MIN_weat = df_test_[df_test_['count_weat'] >= min_term_count]

# df_train_MIN_all = df_train_[df_train_['count_prons'] >= min_term_count]
# df_test_MIN_all = df_test_[df_test_['count_prons'] >= min_term_count]



for spec in ['_all', '_pro', '_weat']:
    
    df_train_MIN = df_train__[df_train__['count'+spec] >= min_term_count]
    df_test_MIN = df_test__[df_test__['count'+spec] >= min_term_count]
    
    # all
    df_train_MIN[['ID', 'text', 'label']].to_pickle('IMDB_training/IMDB_MIN' + spec + '_test')
    df_test_MIN[['ID', 'text', 'label']].to_pickle('IMDB_training/IMDB_MIN' + spec + '_train')
    
    # neutral M+F
    df_train_MIN[['ID', 'text'+spec+'_N', 'label']].to_pickle('IMDB_training/IMDB_MIN_N'+spec+'_train')
    df_test_MIN[['ID', 'text'+spec+'_N', 'label']].to_pickle('IMDB_training/IMDB_MIN_N'+spec+'_test')

    # mixed
    m_tr = df_train_MIN[['ID', 'text'+spec+'_M', 'label']].rename(columns={'text'+spec+'_M': 'text'})
    f_tr = df_train_MIN[['ID', 'text'+spec+'_F', 'label']].rename(columns={'text'+spec+'_F': 'text'})
    tr = m_tr.append(f_tr)
    tr.to_pickle('IMDB_training/IMDB_MIN_mix' + spec + '_train') 
    
    m_te = df_test_MIN[['ID', 'text'+spec+'_M', 'label']].rename(columns={'text'+spec+'_M': 'text'})
    f_te = df_test_MIN[['ID', 'text'+spec+'_F', 'label']].rename(columns={'text'+spec+'_F': 'text'})
    te = m_te.append(f_te)
    te.to_pickle('IMDB_training/IMDB_MIN_mix' + spec + '_test') 
    
    print(tr.shape, te.shape)    

(41644, 3) (41556, 3)
(33356, 3) (32690, 3)
(37024, 3) (36664, 3)


---  
---  
---  
### This is the old Code from EMNLP submission


In [7]:
# resplit 
df = df_train_complete.append(df_test_complete) 
df = df[['Review clean', 'Sentiment']].reset_index()
df

,index,Review clean,Sentiment
0,0,bromwell high is a cartoon comedy it ran at th...,pos
1,1,if you like adult comedy cartoons like south p...,pos
2,2,bromwell high is nothing short of brilliant ex...,pos
3,3,all the world s a stage and its people actors ...,pos
4,4,futz is the only show preserved from the exper...,pos
...,...,...,...
49995,24995,cybertracker is set in los angeles sometime in...,neg
49996,24996,eric phillips don wilson is a secret service a...,neg
49997,24997,plot synopsis los angeles in the future crime ...,neg
49998,24998,oh dear this has to be one of the worst films ...,neg


In [8]:
df_train = df.sample(frac=0.8,random_state=123)
print(df.shape, df_train.shape)
df_train

(50000, 3) (40000, 3)


,index,Review clean,Sentiment
11872,11872,many people like this movie and many more love...,pos
40828,15828,i want to say i liked this film i really do bu...,neg
36400,11400,this is not so much of a review as it is a tes...,pos
5166,5166,indian summer it was very nostalgic for me i f...,pos
30273,5273,the answerno sadly not though miller and the s...,pos
...,...,...,...
33958,8958,extremely good cinematic story of gay embitter...,pos
41359,16359,un funny and un entertaining possibly the wors...,neg
16140,16140,i had no background knowledge of this movie be...,neg
16809,16809,a high school principal keenan wynn with a los...,neg


In [10]:
df_not_train = df[~df.index.isin(df_train.index)]
print(df_not_train.shape)
df_test = df_not_train.sample(frac=0.5,random_state=123)
df_val = df_not_train.drop(df_test.index)
print(df_test.shape, df_val.shape)

(10000, 3)
(5000, 3) (5000, 3)


In [6]:
df_test = df_not_train.sample(frac=0.5,random_state=123)
df_val = df_not_train.drop(df_test.index)
print(df_test.shape, df_val.shape)

(1040, 2) (534, 2)


In [13]:
#df_train.to_excel('IMDb_train.xlsx', engine='xlsxwriter', index=False)
#df_test.to_excel('IMDb_test.xlsx', engine='xlsxwriter', index=False)
#df_val.to_excel('IMDb_val.xlsx', engine='xlsxwriter', index=False)

df_train.to_pickle('IMDb_train')
df_test.to_pickle('IMDb_test')
df_val.to_pickle('IMDb_val')